In [264]:
import geopandas as gpd
import pandas as pd

In [265]:
gdf=gpd.read_file(r"C:\Users\Nathan\OneDrive\Documents\Python_docs\road_acc_proposal\road_dev_proposal\kenya_roads\Kenya_Roads.shx")
gdf.head()


,LINKNO,ROADNO,START_KM,END_KM,LENGTHKM,STRATDESC,ENDDESC,ROADCLASS,CLASS,REGION,WIDTH,LANES,TYPE,SURFTYPE,CONDITION,AADT,geometry
0,A1-1-2-1,A1,0,4,3.53,DB TransNzoia,B4 Marich,A,Primary (Trunk),Rift Valley,7.0,2,Surface Dressing,Paved,Fair,727,"LINESTRING (35.08498 1.21367, 35.08503 1.21375..."
1,A1-1-3-1,A1,0,1,0.73,DB TransNzoia,B4 Marich,A,Primary (Trunk),Rift Valley,7.0,2,Surface Dressing,Paved,Good,727,"LINESTRING (35.09041 1.24291, 35.09048 1.24306..."
2,A1-1-4-1,A1,0,2,1.94,DB TransNzoia,B4 Marich,A,Primary (Trunk),Rift Valley,7.0,2,Surface Dressing,Paved,Fair,727,"LINESTRING (35.09074 1.24922, 35.09072 1.24937..."
3,A1-1-5-1,A1,0,12,12.27,DB TransNzoia,B4 Marich,A,Primary (Trunk),Rift Valley,7.0,2,Surface Dressing,Paved,Good,727,"LINESTRING (35.10177 1.2595, 35.1019 1.25952, ..."
4,A1-1-6-1,A1,0,2,2.19,DB TransNzoia,B4 Marich,A,Primary (Trunk),Rift Valley,7.0,2,Surface Dressing,Paved,Fair,727,"LINESTRING (35.1685 1.27768, 35.1686 1.27759, ..."


In [266]:
gdf['SURFTYPE'].value_counts()

SURFTYPE
Paved      733
Unpaved    240
Name: count, dtype: int64

In [267]:
gdf['STRATDESC'].value_counts()

STRATDESC
DB TransNzoia     50
DB Marsabit       30
A104              28
DB Wajir          26
C80 Wajir         23
                  ..
C16                1
C86 Daranja P.     1
C27 Kisian         1
C65                1
DB Migori          1
Name: count, Length: 190, dtype: int64

In [268]:
gdf['ROADCLASS'].value_counts()

ROADCLASS
A    641
B    332
Name: count, dtype: int64

In [269]:
gdf['ROADNO'].value_counts()

ROADNO
A1       179
A104     116
B9        98
A2        90
A3        60
A109      47
B7        36
B2        34
B8        31
B1        31
B3        28
B4        24
A2S       21
A104N     20
A14       20
B6        18
A104S     17
B5        16
A2N       16
A23       14
A109W      9
A109E      8
A14N       6
A109A      6
A104A      5
A14S       5
B10W       4
B6A        3
B10E       2
B8S        2
B8N        2
B8AN       1
B8AS       1
A1S        1
A1N        1
B1A        1
Name: count, dtype: int64

In [270]:
gdf['START_KM'].unique()

array([0])

In [271]:
gdf=gdf[gdf['ROADCLASS'].isin(['A'])]
gdf.shape

(641, 17)

In [272]:
gdf['REGION'].value_counts()

REGION
Rift Valley      234
Eastern          112
Coast            108
Central           64
Nairobi           42
Western           36
North Eastern     24
Nyanza            21
Name: count, dtype: int64

## THE PLAN
I plan on collapsing the data set to only acquire types of roads from it i will then combine this with accident report data and population projection data in order to 
Prove whether bypass roads are effective
Show which roads would suffer with population increase
whether road condition has any effect on accident rate

In [273]:
pop_df=pd.read_csv('population_projections.csv')
acc_df=pd.read_csv(r'..\cleaned_road_accidents_kenya.csv')
gcp_df=pd.read_csv('county_GCP.csv')

In [274]:
gdf=gdf.drop(columns=['geometry','AADT','TYPE','START_KM','LINKNO'])

In [275]:
gdf.columns

Index(['ROADNO', 'END_KM', 'LENGTHKM', 'STRATDESC', 'ENDDESC', 'ROADCLASS',
       'CLASS', 'REGION', 'WIDTH', 'LANES', 'SURFTYPE', 'CONDITION'],
      dtype='object')

In [276]:
#collapsing gdf per road number
def most_common(series):
    return series.value_counts().idxmax()
collapsed_df = (
    gdf
    .groupby(['ROADNO','REGION'], as_index=False)
    .agg(
        total_length=('LENGTHKM', 'sum'),
        dominant_condition=('CONDITION', most_common),
        works_count=('ROADNO', 'count'),
        width=('WIDTH',most_common),
        lanes=('LANES',most_common),
        surftype=('SURFTYPE',most_common),
        CLASS=('CLASS',most_common),
        rdclass=('ROADCLASS',most_common)
    )
)
collapsed_df.head(20)

,ROADNO,REGION,total_length,dominant_condition,works_count,width,lanes,surftype,CLASS,rdclass
0,A1,Nyanza,161.03,Fair,19,7.0,2,Paved,Primary (Trunk),A
1,A1,Rift Valley,1095.69,Fair,134,7.0,2,Paved,Primary (Trunk),A
2,A1,Western,52.89,Fair,26,7.0,2,Paved,Primary (Trunk),A
3,A104,Central,69.76,Good,10,7.0,2,Paved,Primary (Trunk),A
4,A104,Eastern,28.92,Good,4,7.0,2,Paved,Primary (Trunk),A
5,A104,Nairobi,2.34,Good,2,7.0,2,Paved,Primary (Trunk),A
6,A104,Rift Valley,776.69,Good,90,7.0,2,Paved,Primary (Trunk),A
7,A104,Western,93.10,Fair,10,7.0,2,Paved,Primary (Trunk),A
8,A104A,Rift Valley,12.86,Fair,5,7.0,2,Paved,Primary (Trunk),A
9,A104N,Nairobi,42.46,Fair,17,7.0,2,Paved,Primary (Trunk),A


In [277]:
acc_df=acc_df[acc_df['ROAD CODE'] != 'OTHER']
acc_df['ROAD CODE'].value_counts()

ROAD CODE
A109     123
A104     103
A2        64
A1        21
A3        14
A104S     10
A104E      8
A104N      8
Name: count, dtype: int64

In [278]:
acc_df.groupby('ROAD CODE')['COUNTY'].value_counts()

ROAD CODE  COUNTY      
A1         KAKAMEGA        10
           KISUMU           3
           MIGORI           3
           VIHIGA           3
           HOMA BAY         1
           KISII            1
A104       NAKURU          37
           KIAMBU          18
           KAJIADO         14
           KISUMU          14
           UASIN GISHU      7
           BUSIA            4
           BARINGO          3
           SIAYA            2
           MAKURU           1
           MIGORI           1
           NYANDARUA        1
           VIHIGA           1
A104E      NAIROBI          5
           KIAMBU           3
A104N      KIAMBU           7
           NAIROBI          1
A104S      NAIROBI          7
           KIAMBU           3
A109       MAKUENI         40
           MACHAKOS        24
           NAIROBI         23
           KILIFI          16
           MOMBASA          9
           TAITA TAVETA     7
           KWALE            2
           TAITA-TAVETA     2
A2         KIAMB

In [279]:
kenya_regions = {
    "Nairobi": [
        "Nairobi"
    ],

    "Central": [
        "Kiambu",
        "Murang'a",
        "Nyeri",
        "Kirinyaga",
        "Nyandarua"
    ],

    "Coast": [
        "Mombasa",
        "Kwale",
        "Kilifi",
        "Tana River",
        "Lamu",
        "Taita Taveta"
    ],

    "Eastern": [
        "Machakos",
        "Makueni",
        "Kitui",
        "Embu",
        "Tharaka Nithi",
        "Meru",
        "Isiolo",
        "Marsabit"
    ],

    "North Eastern": [
        "Garissa",
        "Wajir",
        "Mandera"
    ],

    "Nyanza": [
        "Kisumu",
        "Siaya",
        "Homa Bay",
        "Migori",
        "Kisii",
        "Nyamira"
    ],

    "Rift Valley": [
        "Turkana",
        "West Pokot",
        "Samburu",
        "Trans Nzoia",
        "Uasin Gishu",
        "Elgeyo Marakwet",
        "Nandi",
        "Baringo",
        "Laikipia",
        "Nakuru",
        "Narok",
        "Kajiado",
        "Kericho",
        "Bomet"
    ],

    "Western": [
        "Kakamega",
        "Vihiga",
        "Bungoma",
        "Busia"
    ]
}


In [280]:
def county_region_map(row,isAcc):
    if isAcc:
        county=row['COUNTY']
    else:
        county=row['County'] 
    county=county.replace('-', ' ')
    for regions,counties in kenya_regions.items():
        counties=[c.upper() for c in counties]
        #print(counties)
        if county.upper() in counties:
            return regions
acc_df['REGION']=acc_df.apply(lambda row: county_region_map(row, isAcc=True),axis=1)
acc_df['REGION'].value_counts()

REGION
Eastern          70
Central          68
Rift Valley      63
Nairobi          56
Coast            36
Nyanza           25
Western          18
North Eastern     9
Name: count, dtype: int64

In [281]:
acc_df.groupby('ROAD CODE')['REGION'].value_counts()
acc_counts = (
    acc_df
    .groupby(['ROAD CODE', 'REGION'])
    .size()
    .reset_index(name='accident_count')
)
acc_counts.head()

,ROAD CODE,REGION,accident_count
0,A1,Nyanza,8
1,A1,Western,13
2,A104,Central,19
3,A104,Nyanza,17
4,A104,Rift Valley,61


In [282]:
gcp_df['REGION']=gcp_df.apply(lambda row: county_region_map(row,isAcc=False),axis=1)
gcp_df['REGION'].value_counts()

REGION
Rift Valley      14
Eastern           8
Nyanza            6
Coast             6
Central           5
Western           4
North Eastern     3
Nairobi           1
Name: count, dtype: int64

In [283]:
gcp_df[['REGION','County']]

,REGION,County
0,Coast,Mombasa
1,Coast,Kwale
2,Coast,Kilifi
3,Coast,Tana River
4,Coast,Lamu
5,Coast,Taita Taveta
6,North Eastern,Garissa
7,North Eastern,Wajir
8,North Eastern,Mandera
9,Eastern,Marsabit


In [284]:
current_gcp=(
    gcp_df
    .groupby('REGION')
    .agg(
        REGION_GCP=('2020','sum')
    )
)
current_gcp.head(10)

,REGION_GCP
REGION,
Central,1239535
Coast,914152
Eastern,1298393
Nairobi,2685707
North Eastern,165520
Nyanza,907651
Rift Valley,2217134
Western,592588


In [285]:
pop_df['REGION']=pop_df.apply(lambda row: county_region_map(row,isAcc=False),axis=1)
pop_df['REGION'].value_counts()

REGION
Rift Valley      14
Eastern           8
Nyanza            6
Coast             6
Central           5
Western           4
North Eastern     3
Nairobi           1
Name: count, dtype: int64

In [286]:
current_pop=(
    pop_df
    .groupby('REGION')
    .agg(
        REGION_POP=('2020','sum')
    )
)
current_pop.head(10)

,REGION_POP
REGION,
Central,5681427
Coast,4426988
Eastern,7006025
Nairobi,4515607
North Eastern,2552363
Nyanza,6450414
Rift Valley,13063524
Western,5121172


In [287]:
#adding number of accidents to geographic df
collapsed_df = collapsed_df.merge(
    acc_counts,
    left_on=['ROADNO', 'REGION'],
    right_on=['ROAD CODE', 'REGION'],
    how='left'
)
collapsed_df.head(20)

,ROADNO,REGION,total_length,dominant_condition,works_count,width,lanes,surftype,CLASS,rdclass,ROAD CODE,accident_count
0,A1,Nyanza,161.03,Fair,19,7.0,2,Paved,Primary (Trunk),A,A1,8.0
1,A1,Rift Valley,1095.69,Fair,134,7.0,2,Paved,Primary (Trunk),A,NaN,NaN
2,A1,Western,52.89,Fair,26,7.0,2,Paved,Primary (Trunk),A,A1,13.0
3,A104,Central,69.76,Good,10,7.0,2,Paved,Primary (Trunk),A,A104,19.0
4,A104,Eastern,28.92,Good,4,7.0,2,Paved,Primary (Trunk),A,NaN,NaN
5,A104,Nairobi,2.34,Good,2,7.0,2,Paved,Primary (Trunk),A,NaN,NaN
6,A104,Rift Valley,776.69,Good,90,7.0,2,Paved,Primary (Trunk),A,A104,61.0
7,A104,Western,93.10,Fair,10,7.0,2,Paved,Primary (Trunk),A,A104,5.0
8,A104A,Rift Valley,12.86,Fair,5,7.0,2,Paved,Primary (Trunk),A,NaN,NaN
9,A104N,Nairobi,42.46,Fair,17,7.0,2,Paved,Primary (Trunk),A,A104N,1.0


In [289]:
collapsed_df=collapsed_df[collapsed_df['accident_count'].notna()]
collapsed_df.head()

,ROADNO,REGION,total_length,dominant_condition,works_count,width,lanes,surftype,CLASS,rdclass,ROAD CODE,accident_count
0,A1,Nyanza,161.03,Fair,19,7.0,2,Paved,Primary (Trunk),A,A1,8.0
2,A1,Western,52.89,Fair,26,7.0,2,Paved,Primary (Trunk),A,A1,13.0
3,A104,Central,69.76,Good,10,7.0,2,Paved,Primary (Trunk),A,A104,19.0
6,A104,Rift Valley,776.69,Good,90,7.0,2,Paved,Primary (Trunk),A,A104,61.0
7,A104,Western,93.10,Fair,10,7.0,2,Paved,Primary (Trunk),A,A104,5.0


In [294]:
collapsed_df['dominant_condition'].value_counts()

dominant_condition
Fair         6
Good         6
Poor         2
Excellent    1
Name: count, dtype: int64

In [295]:
collapsed_df.shape

(15, 12)

In [297]:
final_df=collapsed_df.drop(columns=['width','lanes','CLASS','rdclass','ROAD CODE'])
final_df.shape

(15, 7)